In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("..")

from src.model import UNet
from src.dataset import get_load_data
from src.loss_function import one_hot_encode
from torch.utils.data import DataLoader
from src.loss_function import energy_loss
from src.loss_function import dice_loss
from src.constants import weights
import torch.nn.functional as F
import torch

In [2]:
unet_model = UNet(num_classes=21)
train, test = get_load_data(root = "../../data", dataset = "VOCSegmentation", download = False)  
train_loader = DataLoader(train, batch_size=4)
imgs, smnts = next(iter(train_loader))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
imgs = imgs.to(device)
smnts = smnts.to(device)
weights = weights.to(device)
unet_model = unet_model.to(device)

In [3]:
import torch
import numpy as np

# img, smnt = train[0]
# img = img.reshape(1, 3, 572, 572)
# smnt = smnt.resize((388, 388))
# smnt = torch.tensor(np.asarray(smnt), dtype = torch.float32).reshape(1, 1, 388, 388)

pred = unet_model(imgs)
smnts_dice = smnts* 255
smnts_dice = torch.where(smnts_dice == 255, 0, smnts_dice)
e_loss = energy_loss(pred, smnts, weight = weights)
e_loss.backward()

In [5]:
pred = unet_model(imgs)

# have to do the few lines below because reasons
smnts_dice = smnts* 255
smnts_dice = torch.where(smnts_dice == 255, 0, smnts_dice)

d_loss = dice_loss(pred, smnts_dice, multiclass= True)
d_loss.backward()

In [6]:
d_loss

tensor(0.9819, device='cuda:0', grad_fn=<RsubBackward1>)